load the pre-processed dataset

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
import nltk
import gensim
import gensim.corpora as corpora
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [ ]:
silhouette_score_list = []
sentence_model = SentenceTransformer("all-MiniLM-L12-v2")
# cluster_model = KMeans(50)
embeddings = sentence_model.encode(df['Text'], show_progress_bar=False)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
vectorizer_model = CountVectorizer(stop_words="english")
# embedding_model = SentenceTransformer("BAAI/bge-base-en-v1.5")
umap_model = UMAP(n_neighbors=13, n_components=15,
                    min_dist=0.0, metric='cosine',random_state=42) # If you want to reproduce the results, at the expense of performance, you can set a random_state in UMAP to prevent any stochastic behavior:
topic_model = BERTopic(umap_model=umap_model,vectorizer_model=vectorizer_model,
                        seed_topic_list = seed_words,
                        top_n_words=5,ctfidf_model=ctfidf_model,min_topic_size=10,calculate_probabilities=True)
topics, probs = topic_model.fit_transform(df['Text'], embeddings)

# Generate `X` and `labels` only for non-outlier topics (as they are technically not clusters)
umap_embeddings = topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

  # Calculate silhouette score
a = silhouette_score(X, labels)
print("silhouette score",a)
documents = pd.DataFrame({"Document": df['Text'],
                          "ID": range(len(df['Text'])),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)]
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
print("coherence score is",coherence)



model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

silhouette score 0.5483653
coherence score is 0.5941087073257405


In [ ]:
# Generate `X` and `labels` only for non-outlier topics (as they are technically not clusters)
umap_embeddings = topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

  # Calculate silhouette score
silhouette_score(X, labels)

0.5483653

In [ ]:
most_representative_words_bertopic_best_1 = topic_model.get_topic_info()
most_representative_words_bertopic_best_1

,Topic,Count,Name,Representation,Representative_Docs
0,-1,950,-1_performance_training_tasks_experiments,"[performance, training, tasks, experiments, data]",[Although the work is aimed at better understa...
1,0,178,0_translation_languages_crosslingual_multilingual,"[translation, languages, crosslingual, multili...",[Our work presents a new dataset based on text...
2,1,88,1_entity_entities_graph_knowledge,"[entity, entities, graph, knowledge, kg]","[Of the conventional fact retrieval pipeline, ..."
3,2,85,2_visual_image_multimodal_scene,"[visual, image, multimodal, scene, images]",[Dataset utilization we have collected 137 dat...
4,3,83,3_qa_question_questions_answer,"[qa, question, questions, answer, table]",[A limitation of rata is always assuming the a...
5,4,76,4_dialogue_conversations_dialog_dialogues,"[dialogue, conversations, dialog, dialogues, m...",[Training data our pre-training data is source...
6,5,71,5_clinical_medical_patients_notes,"[clinical, medical, patients, notes, biomedical]",[There are a few limitations pertaining to the...
7,6,55,6_gender_bias_debiasing_fairness,"[gender, bias, debiasing, fairness, harms]",[Incomplete representation of all demographic ...
8,7,54,7_sentiment_emotion_emotional_wer,"[sentiment, emotion, emotional, wer, reviews]",[Our proposed method is an offline system in w...
9,8,54,8_tuning_hyperparameters_pts_metaretriever,"[tuning, hyperparameters, pts, metaretriever, ...",[The main limitation of our work is the high m...


In [ ]:
most_representative_words_bertopic_best_1.to_csv('df_bertopic.csv',index=False)

# visualization

In [ ]:
topic_model.visualize_topics()


In [ ]:
topic_model.visualize_heatmap()


In [ ]:
# Run the visualization with the original embeddings
topic_model.visualize_documents(df_filtered_doc['Text'], embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(df_filtered_doc['Text'], reduced_embeddings=reduced_embeddings)

In [ ]:
from bertopic import BERTopic

# topic_model = BERTopic()
# topics, _ = topic_model.fit_transform(docs)
topic_distr, _ = topic_model.approximate_distribution(df_filtered_doc['Text'], min_similarity=0)


In [ ]:
# To visualize the probabilities of topic assignment
topic_model.visualize_distribution(probs[0])

# To visualize the topic distributions in a document
topic_model.visualize_distribution(topic_distr[0])


In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(df_filtered_doc['Text'])

topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


100%|██████████| 30/30 [00:00<00:00, 162.79it/s]


In [ ]:
# text based topic tree
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

.
├─language_work_languages_dataset_different
│    ├─transformer_reasoning_parameters_prompt_tasks
│    │    ├─■──tables_table_tableqa_cells_ctbls ── Topic: 21
│    │    └─reasoning_transformer_prompt_parameters_transformers
│    │         ├─reasoning_transformer_parameters_transformers_cost
│    │         │    ├─■──zeroshot_fewshot_verbalizers_ss_bots ── Topic: 19
│    │         │    └─reasoning_transformer_parameters_transformers_nat
│    │         │         ├─transformer_parameters_transformers_nat_cost
│    │         │         │    ├─■──classification_oov_slash_hypermixer_mlpbased ── Topic: 15
│    │         │         │    └─transformer_nat_transformers_parameters_decoding
│    │         │         │         ├─■──hyperparameters_metaretriever_pts_radius_optimization ── Topic: 5
│    │         │         │         └─■──transformer_nat_transformers_decoding_encoder ── Topic: 11
│    │         │         └─■──reasoning_pangu_symbolic_rule_thinksum ── Topic: 13
│    │         └─■──prompts

In [ ]:
# Run the visualization with the original embeddings
topic_model.visualize_hierarchical_documents(df_filtered_doc['Text'], hierarchical_topics, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_hierarchical_documents(df_filtered_doc['Text'], hierarchical_topics, reduced_embeddings=reduced_embeddings)
